### ❗❗❗ Обязательно к первоочердному выполнению ❗❗❗

In [185]:
!wget https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv

!wget https://gist.github.com/Vs8th/aacb80595d1d6aaa2e31eb735f8bc644/raw/entries.csv

!wget https://gist.github.com/Vs8th/0e827e9a608117345dd6585ab81e8c86/raw/metrics.txt

--2025-02-12 03:14:54--  https://gist.github.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.github.com (gist.github.com)... 140.82.121.3
Connecting to gist.github.com (gist.github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv [following]
--2025-02-12 03:14:54--  https://gist.githubusercontent.com/Vs8th/739269a03f2f4a7396d04d6739da3771/raw/registrations.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14918 (15K) [text/plain]
Saving to: 'registrations.csv'

     0K .......... ....                                       100% 7.13M=0.002s

2025-02-12 03:14:54 (7.13 M

❗❗⚠️*Словари заполненны из допущения, что входные данные (csv) отсортированы по дате, по возрастанию*⚠️❗❗

In [186]:
import csv
from datetime import date, timedelta

registration_date_dict = {}
registration_id_dict = {}
with open('registrations.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    header = next(reader)
    for user_id, entry_date in reader:
        registration_date_dict.setdefault(date.fromisoformat(entry_date), set()).add(int(user_id))
        registration_id_dict[int(user_id)] = date.fromisoformat(entry_date)

entries_date_dict = {}
entries_id_dict = {}
with open('entries.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    header = next(reader)
    for user_id, entry_date in reader:
        entries_date_dict.setdefault(date.fromisoformat(entry_date), set()).add(int(user_id))
        entries_id_dict[int(user_id)] = date.fromisoformat(entry_date)

answers = []
with open('metrics.txt', 'r') as f:
    answers = f.read().split('\n')


def check_answers(value_to_check: float, correct_answer: float):
    try:
        assert value_to_check == correct_answer
    except AssertionError:
        print('Ответы не совпадают')
    else:
        print('Поздравляем, Вы справились!')

### Retention 15 дня (в процентах) для пользователей, зарегистрированных в январе

In [187]:
cohort_start_date = date(2021, 1, 1)
cohort_end_date = date(2021, 2, 1)
registrations_qty = 0
entries_qty = 0

for k, v in registration_date_dict.items():
    if k >= cohort_start_date and k < cohort_end_date:
        registrations_qty += len(v)
        entries_qty += len(v.intersection(entries_date_dict.get(k +  timedelta(days=15), {})))

retention_15_day = round((entries_qty/registrations_qty)*100, 5)

**Проверка:**

In [188]:
check_answers(retention_15_day, float(answers[0]))

Поздравляем, Вы справились!


### Rolling-retention 30 дня (в процентах) для пользователей, зарегистрированных в январе

In [189]:
cohort_dict = {key: value for key, value in registration_id_dict.items() 
                if value >= date(2021, 1, 1) and value < date(2021, 2, 1)}

entr_dict = {key: value for key, value in entries_id_dict.items() 
                if value >= cohort_start_date + timedelta(days=30)}

entries_qty = sum(
    1 for k, v in cohort_dict.items()
    if entr_dict.get(k, v) >= v + timedelta(days=30)
)

rolling_retention = round((entries_qty/len(cohort_dict))*100, 5)

**Проверка**

In [190]:
check_answers(rolling_retention, float(answers[1]))

Поздравляем, Вы справились!


### Lifetime по всем пользователям, посчитанный как интеграл от n-day retention

In [191]:
total = sum(len(v) for v in entries_date_dict.values())
lifetime = round((total/len(registration_id_dict)), 5)

**Проверка**

In [207]:
check_answers(lifetime, float(answers[2]))

Поздравляем, Вы справились!


### Churn rate 29 дня (в долях), посчитанный по всем пользователям

In [193]:
user_left_qty = sum(1 for k,v in registration_id_dict.items() 
                    if entries_id_dict[k] < v + timedelta(days=29))
churn_29 = round(user_left_qty / len(registration_id_dict), 5)

**Проверка**

In [208]:
check_answers(churn_29, float(answers[3]))

Поздравляем, Вы справились!


### Mau, Wau, Dau за последний месяц/неделю/день записей

In [195]:
bound_date = date(2021,12,1)
releted_set = set.union(*(v for k, v in entries_date_dict.items() if k.month == bound_date.month))
dec_mau = len(releted_set)

Проверка dec_**Mau**

In [209]:
check_answers(dec_mau, float(answers[4]))

Поздравляем, Вы справились!


In [197]:
bound_date = date(2021,12,4)
releted_set = set.union(*(v for k, v in entries_date_dict.items() if k.month == bound_date.month and k.day > bound_date.day))
dec_wau = len(releted_set)

Проверка dec_**Wau**

In [198]:
check_answers(dec_wau, float(answers[5]))

Поздравляем, Вы справились!


In [199]:
bound_date = date(2021,12,31)
releted_set = set.union(*(v for k, v in entries_date_dict.items() if k == bound_date))
dec_dau = len(releted_set)

Проверка dec_**Dau**

In [200]:
check_answers(dec_dau, float(answers[6]))

Поздравляем, Вы справились!


### Mau, Wau, Dau усредненные

In [201]:
total = 0
for i in range(1, 13):
    total += len(set.union(*(v for k, v in entries_date_dict.items() if k.month == i)))
avg_mau = round(total / 12, 5)

Проверка avg_**Mau**

In [202]:
check_answers(avg_mau, float(answers[7]))

Поздравляем, Вы справились!


In [203]:
import math
result = {}
for k, v in entries_date_dict.items():
    key = math.ceil(k.timetuple().tm_yday / 7)
    result.setdefault(key, set()).update(v)
avg_wau = round(sum(len(v) for v in result.values()) / len(result)-1, 5)

Проверка avg_**Wau**

In [204]:
check_answers(avg_wau, float(answers[8]))

Поздравляем, Вы справились!


In [205]:
total = sum(len(v) for v in entries_date_dict.values())
avg_dau = round(total / 365, 5)

Проверка avg_**Dau**

In [206]:
check_answers(avg_dau, float(answers[9]))

Поздравляем, Вы справились!
